In [1]:
import load
import oed
import figure

In [ ]:
# jsonファイルの読み込み
result = load.load_result_json("../result/Result.json")

gene = result[0]["gene"]

N, M, theta, w_on, w_off, w, g, w_osc, w_nmj = oed.weight(gene)
alpha, x_peak, y_peak, dt, T, f, v, time, tau = oed.constant("setting")
N_, M_, f_inv, T_ = oed.time_constant_step(gene, "setting")

In [ ]:
result_json_file_path = "../result/Result.json"
limit_or_unlimit = "limited"
gene_range = 10
use_function = [
    "connectome",
    "newron_output",
    "Bearing_vs_Turing_bias",
    "Normal_gradient_vs_Turing_bias",
    "Translational_gradient_vs_Turing_bias",
    "trajectory",
    "trajectory_membrane_potential",
]

figure.all_figure_create(
    result_json_file_path, limit_or_unlimit, gene_range, use_function
)